In [1]:
import csv
import pandas as pd
import spacy
import re
import string
import pyLDAvis
import pyLDAvis.sklearn
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from sklearn.decomposition import LatentDirichletAllocation, NMF

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

nlp = spacy.load("en_core_web_sm")

In [2]:
df = pd.read_csv('genshintwittermaintenancetwo.csv')
df = df.drop(columns = ['Unnamed: 0', 'outlinks', 'tcooutlinks'])

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

def cleanurlhashtags(input):

    text = re.sub(r"http\S+", "", input)
    text = re.sub(r"#\S+", "", text)
    return text

def tostring(input):
    return str(input)

df['content'] = df['content'].apply(tostring)
df['content'] = df['content'].apply(cleanurlhashtags)
df['content'] = df['content'].map(alphanumeric).map(punc_lower)
df['content'] = df['content'].fillna(value='')

content = df['content'].to_list()

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = ENGLISH_STOP_WORDS,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(content)

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(content)

lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)

lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=20, random_state=0)

In [3]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

2020-11-14 20:19:02,199 : INFO : NumExpr defaulting to 8 threads.


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
4      0.216597 -0.001519       1        1  8.571051
19     0.283217 -0.091331       2        1  7.059031
7     -0.113065 -0.246829       3        1  5.463247
0     -0.117458  0.113355       4        1  5.371280
12    -0.063916  0.138558       5        1  5.314406
6     -0.090705 -0.268242       6        1  5.140985
5     -0.123270  0.057039       7        1  4.929727
13    -0.087687 -0.179199       8        1  4.846883
2      0.254560 -0.003207       9        1  4.802476
18    -0.076428  0.028432      10        1  4.789425
9      0.064251  0.019401      11        1  4.784925
14    -0.098888 -0.032797      12        1  4.750967
10     0.146607  0.063656      13        1  4.563874
16    -0.060050  0.069203      14        1  4.560198
3     -0.023910  0.098335      15        1  4.264385
8     -0.043577 -0.027166      16        1  4.249226
17     0.051428 -0.039724      17        1  4.221055
11    -0.005253  0.100794      18        1  4.217123
1     -0.000119  0.063400      19        1  4.126606
15    -0.112335  0.137843      20        1  3.973131, topic_info=               Term         Freq        Total Category  logprob  loglift
442         genshin  1788.000000  1788.000000  Default  30.0000  30.0000
557          impact  1222.000000  1222.000000  Default  29.0000  29.0000
175          childe   640.000000   640.000000  Default  28.0000  28.0000
444   genshinimpact   512.000000   512.000000  Default  27.0000  27.0000
432            game   492.000000   492.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
1171           want    28.197542   307.053799  Topic20  -4.3978   0.8378
288            does    18.913962    82.484770  Topic20  -4.7971   1.7529
619            like    25.253817   565.179010  Topic20  -4.5080   0.1175
481           guide    18.600250    78.925291  Topic20  -4.8138   1.7803
164      characters    18.821509   213.884514  Topic20  -4.8020   0.7952

[984 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         5  0.044531     able
0        10  0.111327     able
0        11  0.044531     able
0        14  0.534368     able
0        15  0.022265     able
...     ...       ...      ...
1235      6  0.838577    zhong
1235     15  0.055905    zhong
1236      3  0.081700  zhongli
1236      6  0.359989  zhongli
1236      8  0.556579  zhongli

[2367 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 20, 8, 1, 13, 7, 6, 14, 3, 19, 10, 15, 11, 17, 4, 9, 18, 12, 2, 16])

In [4]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
4      0.238458 -0.021331       1        1  9.133074
19     0.077586  0.135005       2        1  6.217063
13    -0.149962 -0.059827       3        1  5.582488
5      0.038953 -0.116448       4        1  5.457749
14    -0.039561 -0.051409       5        1  5.378709
3      0.069303 -0.024171       6        1  5.267031
12     0.034416 -0.170477       7        1  5.192496
1     -0.037637 -0.119625       8        1  5.105248
7     -0.170962  0.157266       9        1  4.927230
6     -0.161133 -0.003429      10        1  4.802824
15    -0.052824 -0.027348      11        1  4.610493
0     -0.028158 -0.035330      12        1  4.522085
2      0.120486  0.023936      13        1  4.440349
18    -0.139157 -0.061907      14        1  4.364530
8     -0.040095  0.043724      15        1  4.335787
11     0.023634  0.068198      16        1  4.303389
9      0.041774  0.189124      17        1  4.157871
10     0.000764  0.119716      18        1  4.138491
16     0.096779 -0.038100      19        1  4.111998
17     0.077334 -0.007567      20        1  3.951096, topic_info=               Term        Freq       Total Category  logprob  loglift
632            live  168.000000  168.000000  Default  30.0000  30.0000
444   genshinimpact  173.000000  173.000000  Default  29.0000  29.0000
766          paimon   87.000000   87.000000  Default  28.0000  28.0000
687        memepact   79.000000   79.000000  Default  27.0000  27.0000
202            come  146.000000  146.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
220        continue    9.014066   12.483541  Topic20  -4.6073   2.9056
1120          tweet    8.189680   11.549083  Topic20  -4.7032   2.8874
249             cuz    8.324023   17.379016  Topic20  -4.6870   2.4951
442         genshin   14.132177  426.676999  Topic20  -4.1577  -0.1764
557          impact    9.918327  330.096239  Topic20  -4.5117  -0.2738

[988 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         4  0.243782  absolutely
1        19  0.731346  absolutely
3         1  0.133996       abyss
3        13  0.312658       abyss
3        19  0.491319       abyss
...     ...       ...         ...
1236     14  0.039931     zhongli
1236     15  0.055903     zhongli
1236     18  0.055903     zhongli
1236     19  0.007986     zhongli
1236     20  0.007986     zhongli

[2566 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 20, 14, 6, 15, 4, 13, 2, 8, 7, 16, 1, 3, 19, 9, 12, 10, 11, 17, 18])

In [5]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
4     -0.341558 -0.207957       1        1  8.571051
19     0.172838 -0.442897       2        1  7.059031
7     -0.255328 -0.409910       3        1  5.463247
0      0.100923  0.214176       4        1  5.371280
12     0.127841  0.046642       5        1  5.314406
6      0.327823 -0.275446       6        1  5.140985
5      0.040123  0.439241       7        1  4.929727
13    -0.084210 -0.215821       8        1  4.846883
2     -0.392932  0.192767       9        1  4.802476
18     0.217759  0.352153      10        1  4.789425
9      0.374805  0.226011      11        1  4.784925
14    -0.096709  0.062656      12        1  4.750967
10    -0.320616  0.040960      13        1  4.563874
16    -0.095428  0.310889      14        1  4.560198
3     -0.018677 -0.377979      15        1  4.264385
8     -0.240038  0.346437      16        1  4.249226
17     0.126129 -0.206925      17        1  4.221055
11    -0.440912 -0.058865      18        1  4.217123
1      0.371688 -0.094764      19        1  4.126606
15     0.426481  0.058632      20        1  3.973131, topic_info=               Term         Freq        Total Category  logprob  loglift
442         genshin  1788.000000  1788.000000  Default  30.0000  30.0000
557          impact  1222.000000  1222.000000  Default  29.0000  29.0000
175          childe   640.000000   640.000000  Default  28.0000  28.0000
444   genshinimpact   512.000000   512.000000  Default  27.0000  27.0000
432            game   492.000000   492.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
1171           want    28.197542   307.053799  Topic20  -4.3978   0.8378
288            does    18.913962    82.484770  Topic20  -4.7971   1.7529
619            like    25.253817   565.179010  Topic20  -4.5080   0.1175
481           guide    18.600250    78.925291  Topic20  -4.8138   1.7803
164      characters    18.821509   213.884514  Topic20  -4.8020   0.7952

[984 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         5  0.044531     able
0        10  0.111327     able
0        11  0.044531     able
0        14  0.534368     able
0        15  0.022265     able
...     ...       ...      ...
1235      6  0.838577    zhong
1235     15  0.055905    zhong
1236      3  0.081700  zhongli
1236      6  0.359989  zhongli
1236      8  0.556579  zhongli

[2367 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 20, 8, 1, 13, 7, 6, 14, 3, 19, 10, 15, 11, 17, 4, 9, 18, 12, 2, 16])

In [6]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
4      -10.878068   10.538728       1        1  8.571051
19     -60.884026  -75.591820       2        1  7.059031
7       -2.336119  -53.289074       3        1  5.463247
0      -61.385319  -12.914634       4        1  5.371280
12     -28.737743  126.296997       5        1  5.314406
6        1.066606 -123.552811       6        1  5.140985
5       45.140720   -7.947869       7        1  4.929727
13    -140.795975  -98.530548       8        1  4.846883
2       61.531681  -83.240562       9        1  4.802476
18     -72.563057 -140.396881      10        1  4.789425
9       15.984491   63.578186      11        1  4.784925
14    -160.851624   68.945145      12        1  4.750967
10      85.961830   51.922935      13        1  4.563874
16     -99.020836  109.936348      14        1  4.560198
3     -176.764130   -8.681684      15        1  4.264385
8     -117.189880  -37.420200      16        1  4.249226
17     108.829918  -22.946203      17        1  4.221055
11      47.588097  121.934814      18        1  4.217123
1     -106.467834   30.176889      19        1  4.126606
15     -50.613804   59.328842      20        1  3.973131, topic_info=               Term         Freq        Total Category  logprob  loglift
442         genshin  1788.000000  1788.000000  Default  30.0000  30.0000
557          impact  1222.000000  1222.000000  Default  29.0000  29.0000
175          childe   640.000000   640.000000  Default  28.0000  28.0000
444   genshinimpact   512.000000   512.000000  Default  27.0000  27.0000
432            game   492.000000   492.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
1171           want    28.197542   307.053799  Topic20  -4.3978   0.8378
288            does    18.913962    82.484770  Topic20  -4.7971   1.7529
619            like    25.253817   565.179010  Topic20  -4.5080   0.1175
481           guide    18.600250    78.925291  Topic20  -4.8138   1.7803
164      characters    18.821509   213.884514  Topic20  -4.8020   0.7952

[984 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         5  0.044531     able
0        10  0.111327     able
0        11  0.044531     able
0        14  0.534368     able
0        15  0.022265     able
...     ...       ...      ...
1235      6  0.838577    zhong
1235     15  0.055905    zhong
1236      3  0.081700  zhongli
1236      6  0.359989  zhongli
1236      8  0.556579  zhongli

[2367 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 20, 8, 1, 13, 7, 6, 14, 3, 19, 10, 15, 11, 17, 4, 9, 18, 12, 2, 16])